# Домашнее задание по курсу "Методы численного решения задач линейной алгебры"

In [1]:
N_var = 3

Выполнил: студент группы ФН2-32М Матвеев Михаил

Номер варианта (номер в списке в журнале) : 3

## Постановка задачи

Нужно сформировать матрицу размером 10х10 по следующему принципу. В качестве базовой матрицы берется известная матрица, которая получается после дискретизации одномерного оператора Лапласа методом конечных разностей или методом конечных элементов на равномерной сетке:
\begin{equation*}
A_0 = 
\begin{pmatrix}
2 & -1 & 0 & 0 & \cdots & \cdots & \cdots & \cdots & 0 \\
-1 & 2 & -1 & 0 & \cdots & \cdots & \cdots & \cdots & 0 \\
0 & -1 & 2 & -1 & \cdots & \cdots & \cdots & \cdots & 0 \\
 &  &  &  & \ddots &  &  &  &  \\
0 & \cdots & \cdots & \cdots & \cdots & -1 & 2 & -1 & 0 \\
0 & \cdots & \cdots & \cdots & \cdots & 0 & -1 & 2 & -1 \\
0 & \cdots & \cdots & \cdots & \cdots & 0 & 0 & 2 & -1 \\
\end{pmatrix}
\end{equation*}

Для данной матрицы известны аналитические формулы для собственных значений (n=10):
\begin{equation*}
\lambda^{0}_j = 2(1-\cos(\frac{\pi j}{n + 1})), \quad j = 1, \ldots, n
\end{equation*}
и компонент собственных векторов (вектора имеют 2-норму равную 1):
\begin{equation*}
z^{0}_j(k) = \sqrt{\frac{2}{n + 1}}\sin(\frac{\pi j k}{n + 1}), \quad k = 1, \ldots, n
\end{equation*}
Итоговая матрица получается по формулам:
\begin{equation*}
A = A_0 + \delta A,
\end{equation*}


\begin{equation*}
\delta A_{ij} = 
\left\{\begin{matrix}
\dfrac{c}{i + j}, & i \neq j \\ 
0 & i = j
\end{matrix}\right.,
\end{equation*}

\begin{equation*}
c = \frac{N_{var}}{N_{var} + 1}\varepsilon,
\end{equation*}

где $N_{var}$ - номер варианта (совпадает с номером студента в списке в журнале группы), $\varepsilon$  - параметр, значение которого задается далее.

## Подготовка к выполнению заданий

### Импорт необходимых библиотек

In [2]:
import numpy as np
import pandas as pd

### Создание матрицы $A_0$

In [3]:
A_0 = np.identity(10) * 2 + np.eye(10, 10, k = -1) * -1 + np.eye(10, 10, k = 1) * -1

### Код создания матрицы $\delta A$

In [4]:
def construct_delta_A(N_var, eps):
    """
    Алгоритм получения матрицы delta_A
    Ввод: 
    N_var - номер варианта
    eps - параметр
    Вывод:
    delta_A
    """
    par = 10
    delta_A = []
    c = N_var * eps / (N_var + 1)
    for i in range(par):
        temp = []
        for j in range(par):
            temp.append(0 if i == j else c / (i + j))
        delta_A.append(temp)
    return np.array(delta_A)

## Задание №1

Взять матрицу A  для значения $\varepsilon = 0.1$ , убрать последний столбец и сформировать из первых 9 столбцов матрицу $\hat{A}$  размера 10х9. Решить линейную задачу наименьших квадратов для вектора невязки
\begin{equation*}
r = \hat{A} x - b,
\end{equation*}

где вектор b размерности 10х1 нужно получить по следующему алгоритму: выбрать вектор $x_0$  размерности 9х1 и для него вычислить $b = \hat{A} x_0$.

Для решения поставленной задачи использовать QR разложение: для вариантов с четным номером использовать соответствующий алгоритм, основанный на методе вращений Гивенса, для вариантов с нечетным номером – алгоритм, основанный на методе отражений Хаусхолдера. 
После получения решения сделать оценку величины $\dfrac{\left \| x - x_0 \right \|_{2}} {\left \| x_0 \right \|_{2}}$.


In [5]:
eps = 0.1

### Метод отражений Хаусхолдера

#### Описание метода
Преобразованием Хаусхолдера (или отражением) называется матрица вида $P = I - 2 u u^{T}$, где вектор $u$ называется вектором Хаусхолдера и его норма $\left \| u \right \|_2$ = 1. 

Пусть дан вектор $x$. Тогда легко найти отражение $P = I - 2uu^T$, аннулирующее в векторе $x$ все компоненты, кроме первой: $Px = \left [c, 0, \ldots,0  \right ]^T = c \cdot e_1$. По итогу, вектор Хауслендера вычисляется по формуле:
\begin{equation*}
u = x + sign(x_1) \left \| x \right \|_2 e_1
\end{equation*}

#### Описание алгоритма

for i in range(n):
1. подготавливем матрицу P_i (единичная матрица размера m x m, где m - количество строк)
2. в начале каждой итерации выбираем столбец матрицы A (не весь столбец, а все элементы ниже диагонали и диагональный) в качестве вектора x
3. применяем к вектору x метод house для получения вектора Хаусхолдера размера m - i
4. получаем матрицу P_streak размера m - i x m - i
5. добавляем матрицу P_streak в матрицу P_i
6. перемножаем матрицы P_i и R для получения матрицы A_i, у которой под элементом (i, i) - нулевые элементы 
7. домножаем матрицу Q на матрицу P_i (для получения в конце итоговой матрицы Q)

end for

#### Код алгоритма

In [175]:
def house(x):
    u_tilde = np.copy(x)
    u_tilde[0] += np.sign(x[0]) * np.linalg.norm(x)
    return u_tilde / np.linalg.norm(u_tilde)

def qr_householder(A):
    m, n = A.shape
    Q = np.identity(m)
    R = np.copy(A)
    for i in range(n):
        P_i         = np.identity(m) ## 1
        x           = R[i:, i] ## 2
        u           = house(x) ## 3
        P_streak    = np.identity(m - i) - 2 * np.outer(u, u) ## 4
        P_i[i:, i:] = np.copy(P_streak) ## 5
        R           = np.dot(P_i, R) ## 6
        Q           = Q @ P_i ## 7
    return Q[:, :n], R[:n, :] ## Обрезаем для матрицы Q (m-n) столбцов, для матрицы R - (m-n) строк

def qr_householder_effective(A):
    A_temp = np.copy(A)
    l = []
    m, n = A.shape
    for i in range(n):
        x           = A_temp[i:, i] ## 2
        u           = house(x) ## 3
        A_temp[i:m, i:n] -= 2 * np.outer(u, np.matmul(u, A_temp[i:m, i:n]))
        A_temp[i + 1:m, i] = u[1:]
        l.append(u[0])
    return A_temp, l

### Решение задачи

In [186]:
delta_A = construct_delta_A(N_var, 0.1)
A = A_0 + delta_A

amnt_columns = 2
A_hat = np.delete(A, [9 - i for i in range(amnt_columns)], axis = 1).round(2)
x_0 = np.random.rand(10 - amnt_columns) # Выберем случайный вектор x_0
b = np.dot(A_hat, x_0)
b_effective = np.copy(b)

Q, R = qr_householder(A_hat)
A_effective, u_first_elements = qr_householder_effective(A_hat)

for i in range(10 - amnt_columns):
    u = np.hstack((u_first_elements[i],A_effective[i + 1:, i]))
    gamma = -2 * np.inner(u, b_effective[i:])
    b_effective[i:] += gamma * u

assert ((Q @ R - A_hat) < 1e-6).all(), "QR разложение неправильное"

x = np.dot(np.linalg.inv(R) @ Q.T, b) # Решаем линейную задачу наименьших квадратов
x_effective = np.dot(np.linalg.inv(np.triu(A_effective[:-amnt_columns])), b_effective[:-amnt_columns])

value_estimation = np.linalg.norm(x - x_0) / np.linalg.norm(x_0) # Относительная погрешность
result = (f"Решение было получено. \n"
          f"x_0                       = {[round(i, 4) for i in x_0]} \n"
          f"Найдённое решение x       = {[round(i, 4) for i in x]} \n"
          f"Эффективное решение x     = {[round(i, 4) for i in x_effective]} \n"
          f"Относительная погрешность = {value_estimation:.4e}")

### Результаты расчётов

In [187]:
print(result)

Решение было получено. 
x_0                       = [0.7757, 0.8329, 0.2138, 0.469, 0.5086, 0.8021, 0.8877, 0.6029] 
Найдённое решение x       = [0.7757, 0.8329, 0.2138, 0.469, 0.5086, 0.8021, 0.8877, 0.6029] 
Эффективное решение x     = [0.7757, 0.8329, 0.2138, 0.469, 0.5086, 0.8021, 0.8877, 0.6029] 
Относительная погрешность = 6.1524e-16


## Задание №2

Для матрицы $A$ найти все ее собственные значения ($\lambda_j, \quad j = 1,\ldots, 10$) и собственные вектора ($z_j$ , с 2-нормой равной 1) с помощью неявного QR-алгоритма со сдвигом для трех вариантов: $\varepsilon = 10^{-1}, 10^{-3}, 10^{-6}$. 

По итогам расчетов нужно сделать сводную таблицу, в которой указать следующие величины: $\left | \lambda_j - \lambda_j^{0}  \right |$ и $\left \| z_j - z_j^{0} \right \|$  для $j = 1,\ldots,10$ .

In [37]:
eps = [1e-1, 1e-3, 1e-6]